# Data Science Jobs Analyze


Group:
* Anchal
* Angela
* Larissa
* Fabio
* Felipe

This program extracts data from Indeed.ca website to analyze main caracteristics 
of Data Science Job opportunities in Toronto

In [112]:
#################### IMPORT LIBRARIES ####################

In [126]:
#import bs4
import numpy as np
import pandas as pd
import re 
import requests
import datetime
from bs4 import BeautifulSoup

In [127]:
#################### SEARCH ARGUMENTS ####################

In [128]:
input_job = "Data Scientist"
input_quote = False # add quotation marks("") to your input_job
input_city = "" # leave empty if input_city is not specified
input_state = "Canada"
sign = "+"
BASE_URL_indeed =  'https://ca.indeed.com/' #'http://www.indeed.com'


In [129]:
##### Function for Transform searching keywords #####
def transform(input,sign, quote = False):
    syntax = input.replace(" ", sign)
    if quote == True:
        syntax = ''.join(['"', syntax, '"'])
    return(syntax)

In [130]:
################# GENERATING BASE INDEED URL #################

In [131]:

if not input_city: # if (input_city is "")
    url_indeed_list = [ BASE_URL_indeed, '/jobs?q=', transform(input_job, sign, input_quote),
                    '&l=', input_state]
    url_indeed = ''.join(url_indeed_list)
else: # input_city is not ""
    url_indeed_list = [ BASE_URL_indeed, '/jobs?q=', transform(input_job, sign, input_quote),
                    '&l=', transform(input_city, sign), '%2C+', input_state]
    url_indeed = ''.join(url_indeed_list)

print(url_indeed)


https://ca.indeed.com//jobs?q=Data+Scientist&l=Canada


In [132]:
# get the HTML code from the URL
rawcode_indeed = requests.get(url_indeed)

# Choose "lxml" as parser
soup_indeed = BeautifulSoup(rawcode_indeed.text, "lxml")


In [133]:
# total number of results
num_total_indeed = soup_indeed.find(
                        id = 'searchCount').contents[0].split()[-2]

print("total results:")
print(soup_indeed.find(id = 'searchCount').contents[0])
print(soup_indeed.find(id = 'searchCount').contents[0].split()[-2])
num_total_indeed = re.sub("[^0-9]","", num_total_indeed) # remove non-numeric characters in the string
num_total_indeed = int(num_total_indeed)
#print(num_total_indeed)

# total number of pages
num_pages_indeed = int(np.ceil(num_total_indeed/10.0))
#print(num_pages_indeed)

# create an empty dataframe
df_base = pd.DataFrame()
# the date for today
now = datetime.datetime.now()
now_str = now.strftime("%m/%d/%Y")
now_str_name=now.strftime('%m%d%Y')

total results:

        Page 1 of 1,013 jobs
1,013


In [134]:
############ function to clean text before scraping ########### 

In [135]:
def text_cleaner(dirty_text):

    text = dirty_text

    lines = (line.strip() for line in text.splitlines()) # break into lines

    chunks = (phrase.strip() for line in lines for phrase in line.split("  ")) # break multi-headlines into a line each
    
    text = ''.join(chunk for chunk in chunks if chunk).encode('utf-8') # Get rid of all blank lines and ends of line
    
    # Now clean out all of the unicode junk (this line works great!!!)
    
    try:
        text = text.decode('unicode_escape').encode('ascii', 'ignore') # Need this as some websites aren't formatted
    except:                                                            # in a way that this works, can occasionally throw
        return                                                         # an exception
   
    #print(text)
    text = text.decode('utf-8')
    text = re.sub("[^a-zA-Z+3]"," ", text)  # Now get rid of any terms that aren't words (include 3 for d3.js)
                                             # Also include + for C++
    text = re.sub(r"([a-z])([A-Z])", r"\1 \2", text) # Fix spacing issue from merged words
    
    text = text.lower().split()  # Go to lower case and split them apart
    
    
   # stop_words = set(stopwords.words("english")) # Filter out any stop words
   # text = [w for w in text if not w in stop_words]
    
    text = list(set(text)) # Last, just get the set of these. Ignore counts (we are just looking at whether a term existed
                           # or not on the website)
    
    return text

######### CREATING DATA FRAME WITH ALL JOB OPPORTUNITIES URLS #####

In [136]:
######## functions ###########
def Salary(soup):
    try:
        salary = soup.find( attrs = {'class' : 'jobsearch-JobMetadataHeader-item'})
        return  salary.contents
    except:
        return 'Not Informed'
    
def Job_Type(soup):
    try:
        rawcode = requests.get(url)
        soup = BeautifulSoup(rawcode.text, "html.parser")
        job_type = soup.find( attrs = {'class' : 'jobsearch-JobMetadataHeader-item icl-u-xs-mt--xs'})
        return  job_type.contents
    except:
        return 'Not Informed'


In [137]:
########################################
##### Loop for all the total pages #####
########################################

num_pages_indeed = 10 
df_base = pd.DataFrame()
for i in range(1, num_pages_indeed+1):
    # generate the URL
    url = ''.join([url_indeed, '&start=', str(i*10)])
    print(url)

    # get HTML and LXML
    rawcode = requests.get(url)
    soup_lxml = BeautifulSoup(rawcode.text, "lxml")
    
    
    #Extracting the Salary


    # pick out all the "div" with "class="job-row"
    divs = soup_lxml.findAll("div")
    job_divs = [jp for jp in divs if not jp.get('class') is None
                    and 'row' in jp.get('class')]

    
    for job in job_divs:
        try:
            # job id
            id = job.get('data-jk', None)
            
            
            # job link related to job id
            link = 'https://ca.indeed.com/Data-Scientist-jobs-in-Canada?vjk=' + id
                
            # job title
            title = job.find('a', attrs={'data-tn-element': 'jobTitle'}).attrs['title']
            
            # job company
            company = job.find('span', {'class': 'company'}).text.strip()
            
            # job location
            location = job.find('span', {'class': 'location'}).text.strip()
            
            #city, province
            city,province = location.split(',')
            
            r = requests.get(link)
            soup_html = BeautifulSoup(r.text, "html.parser")
            
            # salary
            salary = Salary(soup_html)
            
            # job type
            jobtype = Job_Type(soup_html)

            
            #Extracting the Job Title
            position = soup_html.find(attrs = {'class' : 'icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title'})
            
            
           
            # job Requirements
            try:   
                page = requests.get(link)
                soup = BeautifulSoup(page.text, 'html.parser')
                text  = soup.get_text()
                jobrequirements = text_cleaner(text)
                
            except:
                jobrequirements = 'Forbidden'

            
        except:
            continue
        
        df_base = df_base.append(dict({'Job_ID': id,
                                  'JobTitle': title,
                                  'JobType':jobtype,
                                  'CompanyName': company,
                                  'Salary' : salary ,
                                  'Position':position,
                                  'City': city,
                                  'Province':province,
                                  'JobRequirements':jobrequirements,
                                  'Date': now_str,
                                  'From':'Indeed',
                                  'JobLink':link}),ignore_index=True)
        
print('Finished...')


https://ca.indeed.com//jobs?q=Data+Scientist&l=Canada&start=10
https://ca.indeed.com//jobs?q=Data+Scientist&l=Canada&start=20
https://ca.indeed.com//jobs?q=Data+Scientist&l=Canada&start=30
https://ca.indeed.com//jobs?q=Data+Scientist&l=Canada&start=40
https://ca.indeed.com//jobs?q=Data+Scientist&l=Canada&start=50
https://ca.indeed.com//jobs?q=Data+Scientist&l=Canada&start=60
https://ca.indeed.com//jobs?q=Data+Scientist&l=Canada&start=70
https://ca.indeed.com//jobs?q=Data+Scientist&l=Canada&start=80
https://ca.indeed.com//jobs?q=Data+Scientist&l=Canada&start=90
https://ca.indeed.com//jobs?q=Data+Scientist&l=Canada&start=100
Finished...


In [138]:

df_base.to_csv('out.csv')

In [ ]:
############# READING DATAFRAME AND SCRAPING JOB DESCRIPTION ############

In [1]:
df_base.JobRequirements.values

In [16]:
skill_source = pd.read_excel('ds.xlsx', sheet_name='skill')

kw = pd.read_excel('ds.xlsx', sheet_name='kw')

kw

,kwID,KeyWord,JobTitle,Skill_ID
0,1,mathematics,Data Scientist,6
1,2,Math,Data Scientist,6
2,3,python,Data Scientist,1
3,4,big data,Data Scientist,3
4,5,bigdata,Data Scientist,3
5,6,sql,Java Developer,7
6,7,spring,Java Developer,5
7,8,Not match,Data Scientist,-1


In [93]:

# Functions
def find_skill(skill, jobReq ):
    if skill in jobReq:
        return True
    else:
        return False

In [94]:
df_skill_aux = pd.DataFrame(columns=['Job_ID', 'KeyWord', 'Skill_ID'])


In [95]:
df_skill_aux

,Job_ID,KeyWord,Skill_ID


In [ ]:
z = 0
for index,row in df_base.iterrows():
    k = kw.loc[kw.JobTitle == row["JobTitle"], ['KeyWord', 'Skill_ID']]
    jobReq = row["JobRequirements"]
    
    job_ID = row["Job_ID"]
    
    for idx, skills in k.iterrows():
        skill = skills["KeyWord"]
        skill_ID = skills["Skill_ID"]
        func = find_skill(skill, jobReq)
        print(func)
        if func:
            print(job_ID, skill, skill_ID)
            df_skill_aux.loc[z] = [job_ID, skill, skill_ID]
            z += 1

In [97]:
df_skill_aux

,Job_ID,KeyWord,Skill_ID
0,3625131c23ab13f2,mathematics,6
1,3625131c23ab13f2,python,1
2,79f244142735e51b,python,1
3,9c977136e5f64e8a,python,1
4,85db0c4b9dd0d204,mathematics,6
5,d4a402329a3013c6,mathematics,6
6,e1de8b679753f22a,python,1
7,bdf8400b27b22160,mathematics,6
8,bdf8400b27b22160,python,1
9,2172d8fe11ab66ae,python,1


In [98]:
# Creating df_tmp
df_tmp = pd.DataFrame()


In [ ]:
#df_tmp = df_base.merge(df_skill_aux, how='left', left_on='Job_ID', right_on='ID')
df_tmp = df_base.merge(df_skill_aux, how='left', left_on='Job_ID', right_on='Job_ID')
df_tmp.head()

In [100]:
############# Treating null values #################
values = {'Skill_ID': '-1', 'KeyWord': "Not found"}
df_tmp = df_tmp.fillna(value=values)

In [101]:
df_tmp['Skill_ID'] = df_tmp['Skill_ID'].astype(int)

In [102]:
skill_source

,SkillID,Skill,Category
0,1,Python,Hard
1,2,SQL,Hard
2,3,BigData,Hard
3,4,NumPy,Hard
4,5,Spring,Hard
5,6,Mathematcs,Education
6,7,SQL,Hard
7,8,Statistics,Education
8,-1,Not Identified,Not Idenified


In [106]:
df_jobs = df_tmp.merge(skill_source, left_on='Skill_ID', right_on='Skill_ID', how='left')


KeyError: 'Skill_ID'

In [104]:
df_jobs

NameError: name 'df_jobs' is not defined

In [ ]:
df_jobs = df_jobs.drop_duplicates(subset={'Job_ID','Skill_ID'}, keep='first', inplace=False)

In [ ]:
df_jobs